In [1]:
# ---------- CELL 1: Install & Preflight ----------
# Run this first. It installs cloudflared and Python libs.
# It may take a few minutes (model downloads happen in later cell).

# System packages + cloudflared
!apt-get update -qq
!apt-get install -y -qq wget ca-certificates ffmpeg fonts-noto

# Download and install cloudflared (auto tunnel)
!wget -q -O /tmp/cloudflared.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
!dpkg -i /tmp/cloudflared.deb || true
!apt-get -f install -y -qq

# Python packages (transformers may be large)
!pip install -q yt-dlp moviepy==1.0.3 ffmpeg-python transformers[torch] accelerate sentencepiece soundfile gTTS flask

# Try installing Bark (optional, better TTS). If it fails, code will fallback to gTTS.
try:
    !pip install -q git+https://github.com/suno-ai/bark.git
except Exception as e:
    print("Bark install attempt failed (no problem - gTTS fallback will be used).", e)

# Quick checks
import sys, torch
print("Python:", sys.version.split()[0])
print("Torch:", getattr(torch, "__version__", "not installed"))
print("GPU available:", torch.cuda.is_available())

# Create output folder
import os
os.makedirs('/content/auto_videos', exist_ok=True)
print("Output directory:", "/content/auto_videos")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 127572 files and directories currently installed.)
Preparing to unpack /tmp/cloudflared.deb ...
Unpacking cloudflared (2025.10.1) over (2025.10.1) ...
Setting up cloudflared (2025.10.1) ...
Processing triggers for man-db (2.10.2-1) ...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Python: 3.12.12
Torch: 2.8.0+cu126
GPU available: True
Output directory: /content/auto_videos


In [2]:
# ---------- CELL 2: Production Queue Server + cloudflared ----------
# Paste & run this after CELL 1. Keep this notebook open while testing.

import os, time, uuid, json, subprocess, threading, queue, re
from flask import Flask, request, jsonify
from transformers import pipeline
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip, concatenate_videoclips, ColorClip
from gtts import gTTS

# CONFIG
OUTPUT_DIR = "/content/auto_videos"
os.makedirs(OUTPUT_DIR, exist_ok=True)
PORT = 5000
WIDTH, HEIGHT, FPS = 1080, 1920, 25
FACTS_COUNT = 5
USERNAME_WM = "darktruths.hub007"

# --------- Text generation (flan-t5-large) ----------
print("Loading text model (flan-t5-large). This may download ~1GB and take several minutes.")
# device mapping: GPU if available else CPU (-1)
device = 0 if __import__('torch').cuda.is_available() else -1
text_gen = pipeline('text2text-generation', model='google/flan-t5-large', device=device)

def generate_facts(topic, n=5):
    prompt = f"Write {n} short, creepy, factual one-line facts about {topic}. Each fact 6-16 words, punchy, suitable for a short social video. Return as a JSON array or newline separated list."
    out = text_gen(prompt, max_length=256, do_sample=True, top_p=0.95, num_return_sequences=1)[0]['generated_text']
    # Try parse JSON array
    try:
        arr = json.loads(out)
        if isinstance(arr, list):
            return [s.strip() for s in arr][:n]
    except:
        # fallback: split heuristically
        parts = [p.strip() for p in out.replace('\n',' . ').split('.') if p.strip()]
        facts = []
        for p in parts:
            if len(facts) >= n: break
            if len(p.split()) >= 3:
                facts.append(p)
        while len(facts) < n:
            facts.append(f"Creepy fact about {topic} {len(facts)+1}")
        return facts

# --------- TTS: Bark preferred, fallback to gTTS ----------
BARK_AVAILABLE = False
try:
    from bark import generate_audio, preload_models, SAMPLE_RATE
    import soundfile as sf
    try:
        print("Preloading Bark models (may download tens to hundreds MB).")
        preload_models()
    except Exception as e:
        print("Bark preload warning:", e)
    BARK_AVAILABLE = True
    print("Bark detected - will use Bark for TTS.")
except Exception as e:
    print("Bark not available - will use gTTS fallback. (This is fine.)", e)
    BARK_AVAILABLE = False

def synthesize_bark(text, out_path, history_prompt="whisper"):
    try:
        wav = generate_audio(text=text, history_prompt=history_prompt)
        import soundfile as sf
        sf.write(out_path, wav, SAMPLE_RATE)
        return out_path
    except Exception as e:
        print("Bark generation failed:", e)
        return None

def synthesize_gtts(text, out_path):
    tts = gTTS(text=text, lang='en')
    tts.save(out_path)
    return out_path

def synthesize_voice(text, out_path):
    # Add small pauses for natural pacing
    utter = ". ".join([s.strip() for s in text.split('.') if s.strip()]) + "."
    if BARK_AVAILABLE:
        out = synthesize_bark(utter, out_path, history_prompt="whisper")
        if out:
            return out
    return synthesize_gtts(utter, out_path)

# --------- B-roll downloader (yt-dlp, searches Creative Commons) ----------
def download_youtube_cc(query, max_count=1, max_seconds=8):
    # Uses ytsearch with "creative commons" appended to encourage CC picks
    import subprocess, glob
    cmd = [
        'yt-dlp',
        f'ytsearch{max_count}:{query} creative commons',
        '--no-playlist',
        '--restrict-filenames',
        '--format', 'mp4',
        '--output', os.path.join(OUTPUT_DIR, 'ytclip_%(id)s.%(ext)s')
    ]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        print("yt-dlp download issue:", e)
    files = sorted([f for f in os.listdir(OUTPUT_DIR) if f.startswith('ytclip_') and f.endswith('.mp4')])
    final = []
    for f in files[:max_count]:
        full = os.path.join(OUTPUT_DIR, f)
        try:
            clip = VideoFileClip(full)
            dur = min(max_seconds, clip.duration)
            outtrim = full.replace('.mp4', '_trim.mp4')
            clip.subclip(0, dur).write_videofile(outtrim, fps=FPS, codec='libx264', audio_codec='aac', threads=2, verbose=False, logger=None)
            clip.close()
            final.append(outtrim)
        except Exception as e:
            print("Error trimming clip:", e)
    return final

# --------- Video builder (vertical 1080x1920) ----------
def build_vertical_video(facts, voice_path, broll_clips, out_path, username=USERNAME_WM):
    # background clips
    bg_clips = []
    for c in broll_clips:
        try:
            vc = VideoFileClip(c)
            vc = vc.resize(height=HEIGHT)
            if vc.w < WIDTH:
                vc = vc.resize(width=WIDTH)
            vc = vc.subclip(0, min(8, vc.duration))
            bg_clips.append(vc)
        except Exception as e:
            print("bg clip load error:", e)
    if not bg_clips:
        bg = ColorClip((WIDTH, HEIGHT), color=(12,12,16)).set_duration(20)
    else:
        bg = concatenate_videoclips(bg_clips, method='compose').loop(duration=60)

    # overlays: facts sequentially
    per = 5.5
    start = 0.8
    overlays = []
    for fact in facts:
        txt = TextClip(fact, fontsize=64, font='DejaVuSans', method='caption', size=(WIDTH-140, None), align='center')
        txt = txt.set_position(('center', HEIGHT*0.14)).set_start(start).set_duration(per)
        overlays.append(txt)
        start += per

    watermark = TextClip(username, fontsize=28, font='DejaVuSans').set_pos(('right', 'bottom')).set_duration(start + 1)
    audio = AudioFileClip(voice_path)
    duration = audio.duration
    final = CompositeVideoClip([bg] + overlays + [watermark], size=(WIDTH, HEIGHT)).set_duration(duration)
    final = final.set_audio(audio)
    final.write_videofile(out_path, fps=FPS, codec='libx264', audio_codec='aac', threads=4)
    # cleanup
    try:
        for c in bg_clips: c.close()
    except:
        pass
    return out_path

# --------- Job queue worker ----------
job_q = queue.Queue()
jobs = {}  # jobid -> info dict

def worker():
    while True:
        jobid, topic = job_q.get()
        jobs[jobid]['status'] = 'running'
        try:
            facts = generate_facts(topic, n=FACTS_COUNT)
            voice_file = os.path.join(OUTPUT_DIR, f"voice_{jobid}.wav")
            synthesize_voice('. '.join(facts) + '.', voice_file)
            # download b-roll for two queries (dark fog + abandoned)
            broll = download_youtube_cc('dark fog cinematic', max_count=1, max_seconds=8)
            broll += download_youtube_cc('abandoned building cinematic', max_count=1, max_seconds=8)
            out_file = os.path.join(OUTPUT_DIR, f'creepy_{jobid}.mp4')
            build_vertical_video(facts, voice_file, broll, out_file)
            jobs[jobid]['status'] = 'done'
            jobs[jobid]['result'] = out_file
            jobs[jobid]['finished'] = time.time()
        except Exception as e:
            jobs[jobid]['status'] = 'error'
            jobs[jobid]['error'] = str(e)
        job_q.task_done()

# start single worker thread
threading.Thread(target=worker, daemon=True).start()

# --------- Flask app: enqueue + status endpoints ----------
app = Flask(__name__)

@app.route("/generate", methods=["POST"])
def enqueue():
    data = request.get_json(force=True, silent=True) or {}
    topic = data.get('topic','creepy facts')
    jobid = str(uuid.uuid4())[:8]
    jobs[jobid] = {"status":"queued","topic":topic,"created":time.time()}
    job_q.put((jobid, topic))
    return jsonify({"status":"queued","jobid":jobid,"poll":f"/status/{jobid}"})

@app.route("/status/<jobid>", methods=["GET"])
def status(jobid):
    info = jobs.get(jobid)
    if not info:
        return jsonify({"error":"unknown jobid"}), 404
    return jsonify(info)

@app.route("/", methods=["GET"])
def root():
    return jsonify({"status":"alive","info":"Production queue API. POST /generate with {'topic':'...'}"})

# --------- Start Flask + cloudflared tunnel (auto) ----------
def start_flask():
    app.run(host='0.0.0.0', port=PORT)

def start_cloudflared_and_print():
    cmd = ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"]
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    public_url = None
    for _ in range(400):
        line = popen.stdout.readline()
        if not line:
            time.sleep(0.1)
            continue
        print(line.strip())
        # look for trycloudflare URL
        if "trycloudflare.com" in line or "trycloudflare" in line:
            m = re.search(r'(https?://[^\s]+trycloudflare[^\s]*)', line)
            public_url = m.group(1) if m else line.strip()
            print("\n✅ PUBLIC URL:", public_url)
            print("Use this to POST /generate and GET /status/<jobid>\n")
            break

# run both
threading.Thread(target=start_flask, daemon=True).start()
time.sleep(1)
threading.Thread(target=start_cloudflared_and_print, daemon=True).start()

  warnings.warn(



Loading text model (flan-t5-large). This may download ~1GB and take several minutes.


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Preloading Bark models (may download tens to hundreds MB).
Bark preload warning: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([numpy.core.multiarray.scalar])` or the `torch.serialization.safe_globals([numpy.core.multiarray.scalar])` context manager to allowlist this global if you trust this class/funct

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [3]:
# =========================
# 🌐 CLOUDFLARED TUNNEL SETUP
# =========================
import subprocess, time, re

# Install cloudflared if not already installed
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

PORT = 5000  # same port as Flask API

def start_tunnel():
    print("Starting Cloudflared tunnel...")
    process = subprocess.Popen(["./cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--logfile", "cloudflared.log"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    time.sleep(5)

    # read log for the public URL
    with open("cloudflared.log") as f:
        logs = f.read()
        urls = re.findall(r"https://.*?trycloudflare\.com", logs)
        if urls:
            public_url = urls[-1]
            print("\n✅ Tunnel active!")
            print("Public API URL:", public_url)
            print("\nPOST to: " + public_url + "/generate")
            return public_url
        else:
            print("⚠️ Cloudflared didn't return a URL. Restarting...")
            process.kill()
            time.sleep(3)
            return start_tunnel()

public_url = start_tunnel()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  28.9M      0  0:00:01  0:00:01 --:--:--  143M
Starting Cloudflared tunnel...

✅ Tunnel active!
Public API URL: https://mountains-tend-herbal-remarkable.trycloudflare.com

POST to: https://mountains-tend-herbal-remarkable.trycloudflare.com/generate
